# Fedbiomed Researcher

Use for developing (autoreloads changes made across packages)

In [1]:
%load_ext autoreload
%autoreload 2

Loading tensorboard extension

In [2]:
%load_ext tensorboard

## Setting the client up
It is necessary to previously configure a node:
1. `./scripts/fedbiomed_run node add`
  * Select option 2 (default) to add MNIST to the client
  * Confirm default tags by hitting "y" and ENTER
  * Pick the folder where MNIST is downloaded (this is due torch issue https://github.com/pytorch/vision/issues/3549)
  * Data must have been added (if you get a warning saying that data must be unique is because it's been already added)
  
2. Check that your data has been added by executing `./scripts/fedbiomed_run node list`
3. Run the node using `./scripts/fedbiomed_run node run`. Wait until you get `Connected with result code 0`. it means you are online.

## Create an experiment to train a model on the data found

Declare a torch.nn MyTrainingPlan class to send for training on the node

In [3]:
from fedbiomed.researcher.environ import TMP_DIR
import tempfile
tmp_dir_model = tempfile.TemporaryDirectory(dir=TMP_DIR+'/')
model_file = tmp_dir_model.name + '/class_export_mnist.py'

Note : write **only** the code to export in the following cell

In [4]:
%%writefile "$model_file"

import torch
import torch.nn as nn
from fedbiomed.common.torchnn import TorchTrainingPlan
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Here we define the model to be used. 
# You can use any class name (here 'Net')
class MyTrainingPlan(TorchTrainingPlan):
    def __init__(self):
        super(MyTrainingPlan, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)
        
        # Here we define the custom dependencies that will be needed by our custom Dataloader
        # In this case, we need the torch DataLoader classes
        # Since we will train on MNIST, we need datasets and transform from torchvision
        deps = ["from torchvision import datasets, transforms",
               "from torch.utils.data import DataLoader"]
        self.add_dependency(deps)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

    def training_data(self, batch_size = 48):
        # Custom torch Dataloader for MNIST data
        transform = transforms.Compose([transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))])
        dataset1 = datasets.MNIST(self.dataset_path, train=True, download=False, transform=transform)
        train_kwargs = {'batch_size': batch_size, 'shuffle': True}
        data_loader = torch.utils.data.DataLoader(dataset1, **train_kwargs)
        return data_loader
    
    def training_step(self, data, target):
        output = self.forward(data)
        loss   = torch.nn.functional.nll_loss(output, target)
        return loss


Writing /user/scansiz/home/Desktop/Inria/dev/fedbiomed/var/tmp/tmp9z0t8pkl/class_export_mnist.py


This group of arguments correspond respectively:
* `model_args`: a dictionary with the arguments related to the model (e.g. number of layers, features, etc.). This will be passed to the model class on the client side.
* `training_args`: a dictionary containing the arguments for the training routine (e.g. batch size, learning rate, epochs, etc.). This will be passed to the routine on the client side.

**NOTE:** typos and/or lack of positional (required) arguments will raise error. 🤓

In [5]:
model_args = {}

training_args = {
    'batch_size': 48, 
    'lr': 1e-3, 
    'epochs': 3, 
    'dry_run': False,  
    'batch_maxnum': 100 # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
}

Define an experiment
- search nodes serving data for these `tags`, optionally filter on a list of client ID with `clients`
- run a round of local training on nodes with model defined in `model_path` + federation with `aggregator`
- run for `rounds` rounds, applying the `client_selection_strategy` between the rounds

In [6]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['#MNIST', '#dataset']
rounds = 2

exp = Experiment(tags=tags,
                 #clients=None,
                 model_path=model_file,
                 model_args=model_args,
                 model_class='MyTrainingPlan',
                 training_args=training_args,
                 rounds=rounds,
                 aggregator=FedAverage(),
                 client_selection_strategy=None,
                 tensorboard=True
                )

2021-09-29 17:25:01,855 fedbiomed INFO - Messaging researcher_46a0da2b-2e99-4316-b964-b467d343432b successfully connected to the message broker, object = <fedbiomed.common.messaging.Messaging object at 0x7f40b1ece3d0>
2021-09-29 17:25:01,866 fedbiomed INFO - Searching for clients with data tags: ['#MNIST', '#dataset']
2021-09-29 17:25:01,916 fedbiomed INFO - message received:{'researcher_id': 'researcher_46a0da2b-2e99-4316-b964-b467d343432b', 'success': True, 'databases': [{'name': 'MNIST', 'data_type': 'default', 'tags': ['#MNIST', '#dataset'], 'description': 'MNIST database', 'shape': [60000, 1, 28, 28], 'dataset_id': 'dataset_da9a5db6-9dda-475e-b920-43f8d1f54377'}], 'count': 1, 'client_id': 'client_b1afd615-e4d4-489f-9ded-bd8db8ff2137', 'command': 'search'}
2021-09-29 17:25:01,918 fedbiomed INFO - message received:{'researcher_id': 'researcher_46a0da2b-2e99-4316-b964-b467d343432b', 'success': True, 'databases': [{'name': 'MNIST', 'data_type': 'default', 'tags': ['#MNIST', '#dataset'

Let's start the experiment.

By default, this function doesn't stop until all the `rounds` are done for all the clients

In [7]:
exp.run()

2021-09-29 17:25:12,244 fedbiomed INFO - Sampled clients in round 0 ['client_b1afd615-e4d4-489f-9ded-bd8db8ff2137', 'client_2a34fb5d-7c54-469f-b034-51062b388dc1']
2021-09-29 17:25:12,244 fedbiomed INFO - Send message to client client_b1afd615-e4d4-489f-9ded-bd8db8ff2137 - {'researcher_id': 'researcher_46a0da2b-2e99-4316-b964-b467d343432b', 'job_id': '165cee09-a38f-4214-b88c-011befcb2ff3', 'training_args': {'batch_size': 48, 'lr': 0.001, 'epochs': 3, 'dry_run': False, 'batch_maxnum': 100}, 'model_args': {}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2021/09/29/my_model_596a43d3-0253-4ab6-a5eb-52107677b45a.py', 'params_url': 'http://localhost:8844/media/uploads/2021/09/29/my_model_92c75bb5-6dad-413c-92b8-909e9c05ef38.pt', 'model_class': 'MyTrainingPlan', 'training_data': {'client_b1afd615-e4d4-489f-9ded-bd8db8ff2137': ['dataset_da9a5db6-9dda-475e-b920-43f8d1f54377']}}
2021-09-29 17:25:12,245 fedbiomed DEBUG - researcher_46a0da2b-2e99-4316-b964-b467d343432b
2021

2021-09-29 17:25:31,864 fedbiomed INFO - Round: 0 Node: client_b1afd615-e4d4-489f-9ded-bd8db8ff2137 - Train Epoch: 3 [960/60000]	Loss: 0.076419
2021-09-29 17:25:32,302 fedbiomed INFO - Round: 0 Node: client_2a34fb5d-7c54-469f-b034-51062b388dc1 - Train Epoch: 3 [1440/60000]	Loss: 0.120297
2021-09-29 17:25:32,816 fedbiomed INFO - Round: 0 Node: client_b1afd615-e4d4-489f-9ded-bd8db8ff2137 - Train Epoch: 3 [1440/60000]	Loss: 0.327884
2021-09-29 17:25:33,197 fedbiomed INFO - Round: 0 Node: client_2a34fb5d-7c54-469f-b034-51062b388dc1 - Train Epoch: 3 [1920/60000]	Loss: 0.082900
2021-09-29 17:25:33,759 fedbiomed INFO - Round: 0 Node: client_b1afd615-e4d4-489f-9ded-bd8db8ff2137 - Train Epoch: 3 [1920/60000]	Loss: 0.159018
2021-09-29 17:25:34,076 fedbiomed INFO - Round: 0 Node: client_2a34fb5d-7c54-469f-b034-51062b388dc1 - Train Epoch: 3 [2400/60000]	Loss: 0.197126
2021-09-29 17:25:34,560 fedbiomed INFO - Round: 0 Node: client_b1afd615-e4d4-489f-9ded-bd8db8ff2137 - Train Epoch: 3 [2400/60000]	L

2021-09-29 17:25:55,601 fedbiomed INFO - Round: 1 Node: client_b1afd615-e4d4-489f-9ded-bd8db8ff2137 - Train Epoch: 1 [4320/60000]	Loss: 0.159965
2021-09-29 17:25:56,113 fedbiomed INFO - Round: 1 Node: client_2a34fb5d-7c54-469f-b034-51062b388dc1 - Train Epoch: 1 [3840/60000]	Loss: 0.118337
2021-09-29 17:25:56,631 fedbiomed INFO - Round: 1 Node: client_b1afd615-e4d4-489f-9ded-bd8db8ff2137 - Train Epoch: 2 [0/60000]	Loss: 0.139654
2021-09-29 17:25:56,957 fedbiomed INFO - Round: 1 Node: client_2a34fb5d-7c54-469f-b034-51062b388dc1 - Train Epoch: 1 [4320/60000]	Loss: 0.332354
2021-09-29 17:25:57,353 fedbiomed INFO - Round: 1 Node: client_b1afd615-e4d4-489f-9ded-bd8db8ff2137 - Train Epoch: 2 [480/60000]	Loss: 0.129357
2021-09-29 17:25:58,038 fedbiomed INFO - Round: 1 Node: client_2a34fb5d-7c54-469f-b034-51062b388dc1 - Train Epoch: 2 [0/60000]	Loss: 0.072352
2021-09-29 17:25:58,123 fedbiomed INFO - Round: 1 Node: client_b1afd615-e4d4-489f-9ded-bd8db8ff2137 - Train Epoch: 2 [960/60000]	Loss: 0.

To display current values please click refresh button on the TensorBoard screen

In [8]:
%tensorboard --logdir '../var/tmp/tensorboard'

2021-09-30 10:43:53,026 fedbiomed INFO - message received:{'researcher_id': 'researcher_46a0da2b-2e99-4316-b964-b467d343432b', 'success': True, 'databases': [{'name': 'MNIST', 'data_type': 'default', 'tags': ['#MNIST', '#dataset'], 'description': 'MNIST database', 'shape': [60000, 1, 28, 28], 'dataset_id': 'dataset_da9a5db6-9dda-475e-b920-43f8d1f54377'}], 'count': 1, 'client_id': 'client_b1afd615-e4d4-489f-9ded-bd8db8ff2137', 'command': 'search'}
2021-09-30 10:43:53,028 fedbiomed INFO - message received:{'researcher_id': 'researcher_46a0da2b-2e99-4316-b964-b467d343432b', 'success': True, 'databases': [{'name': 'MNIST', 'data_type': 'default', 'tags': ['#MNIST', '#dataset'], 'description': 'MNIST database', 'shape': [60000, 1, 28, 28], 'dataset_id': 'dataset_655ec19c-3a4c-4b23-9e39-c91deba0a3e3'}], 'count': 1, 'client_id': 'client_2a34fb5d-7c54-469f-b034-51062b388dc1', 'command': 'search'}
2021-09-30 10:45:09,783 fedbiomed INFO - message received:{'researcher_id': 'researcher_46a0da2b-2

2021-09-30 10:50:36,528 fedbiomed INFO - Round: 0 Node: client_2a34fb5d-7c54-469f-b034-51062b388dc1 - Train Epoch: 2 [1440/60000]	Loss: 0.255010
2021-09-30 10:50:37,112 fedbiomed INFO - Round: 0 Node: client_b1afd615-e4d4-489f-9ded-bd8db8ff2137 - Train Epoch: 2 [1440/60000]	Loss: 0.090004
2021-09-30 10:50:37,433 fedbiomed INFO - Round: 0 Node: client_2a34fb5d-7c54-469f-b034-51062b388dc1 - Train Epoch: 2 [1920/60000]	Loss: 0.077580
2021-09-30 10:50:38,014 fedbiomed INFO - Round: 0 Node: client_b1afd615-e4d4-489f-9ded-bd8db8ff2137 - Train Epoch: 2 [1920/60000]	Loss: 0.263381
2021-09-30 10:50:38,209 fedbiomed INFO - Round: 0 Node: client_2a34fb5d-7c54-469f-b034-51062b388dc1 - Train Epoch: 2 [2400/60000]	Loss: 0.162341
2021-09-30 10:50:38,835 fedbiomed INFO - Round: 0 Node: client_b1afd615-e4d4-489f-9ded-bd8db8ff2137 - Train Epoch: 2 [2400/60000]	Loss: 0.138241
2021-09-30 10:50:39,013 fedbiomed INFO - Round: 0 Node: client_2a34fb5d-7c54-469f-b034-51062b388dc1 - Train Epoch: 2 [2880/60000]	

2021-09-30 10:51:11,502 fedbiomed INFO - Round: 0 Node: client_b1afd615-e4d4-489f-9ded-bd8db8ff2137 - Train Epoch: 1 [0/60000]	Loss: 2.319746
2021-09-30 10:51:13,403 fedbiomed INFO - Round: 0 Node: client_b1afd615-e4d4-489f-9ded-bd8db8ff2137 - Train Epoch: 1 [480/60000]	Loss: 1.089135
2021-09-30 10:51:14,158 fedbiomed INFO - Round: 0 Node: client_2a34fb5d-7c54-469f-b034-51062b388dc1 - Train Epoch: 1 [480/60000]	Loss: 1.516865
2021-09-30 10:51:15,398 fedbiomed INFO - Round: 0 Node: client_2a34fb5d-7c54-469f-b034-51062b388dc1 - Train Epoch: 1 [960/60000]	Loss: 0.630912
2021-09-30 10:51:15,530 fedbiomed INFO - Round: 0 Node: client_b1afd615-e4d4-489f-9ded-bd8db8ff2137 - Train Epoch: 1 [960/60000]	Loss: 0.861029
2021-09-30 10:51:16,265 fedbiomed INFO - Round: 0 Node: client_2a34fb5d-7c54-469f-b034-51062b388dc1 - Train Epoch: 1 [1440/60000]	Loss: 0.789396
2021-09-30 10:51:16,392 fedbiomed INFO - Round: 0 Node: client_b1afd615-e4d4-489f-9ded-bd8db8ff2137 - Train Epoch: 1 [1440/60000]	Loss: 0

2021-09-30 10:55:26,858 fedbiomed INFO - message received:{'researcher_id': 'researcher_46a0da2b-2e99-4316-b964-b467d343432b', 'success': True, 'databases': [{'name': 'MNIST', 'data_type': 'default', 'tags': ['#MNIST', '#dataset'], 'description': 'MNIST database', 'shape': [60000, 1, 28, 28], 'dataset_id': 'dataset_655ec19c-3a4c-4b23-9e39-c91deba0a3e3'}], 'count': 1, 'client_id': 'client_2a34fb5d-7c54-469f-b034-51062b388dc1', 'command': 'search'}
2021-09-30 10:56:03,077 fedbiomed INFO - message received:{'researcher_id': 'researcher_46a0da2b-2e99-4316-b964-b467d343432b', 'job_id': '778279c2-1740-4a54-bad5-e96ae8e5c977', 'success': True, 'client_id': 'client_2a34fb5d-7c54-469f-b034-51062b388dc1', 'dataset_id': 'dataset_655ec19c-3a4c-4b23-9e39-c91deba0a3e3', 'params_url': 'http://localhost:8844/media/uploads/2021/09/30/node_params_c909ea71-65b8-43f9-a676-59b82c83ec5e.pt', 'timing': {'rtime_training': 25.685859377961606, 'ptime_training': 96.03053559100044}, 'msg': '', 'command': 'train'}

2021-09-30 13:37:58,091 fedbiomed INFO - message received:{'researcher_id': 'researcher_46a0da2b-2e99-4316-b964-b467d343432b', 'success': True, 'databases': [{'name': 'MNIST', 'data_type': 'default', 'tags': ['#MNIST', '#dataset'], 'description': 'MNIST database', 'shape': [60000, 1, 28, 28], 'dataset_id': 'dataset_655ec19c-3a4c-4b23-9e39-c91deba0a3e3'}], 'count': 1, 'client_id': 'client_2a34fb5d-7c54-469f-b034-51062b388dc1', 'command': 'search'}
2021-09-30 13:38:53,961 fedbiomed INFO - message received:{'researcher_id': 'researcher_46a0da2b-2e99-4316-b964-b467d343432b', 'job_id': 'f660c08c-b703-4c5d-8e14-87fa16eea926', 'success': True, 'client_id': 'client_2a34fb5d-7c54-469f-b034-51062b388dc1', 'dataset_id': 'dataset_655ec19c-3a4c-4b23-9e39-c91deba0a3e3', 'params_url': 'http://localhost:8844/media/uploads/2021/09/30/node_params_8a193149-4966-485b-a794-573fab32b85a.pt', 'timing': {'rtime_training': 45.19463989592623, 'ptime_training': 142.4015328839996}, 'msg': '', 'command': 'train'}


2021-09-30 13:44:11,444 fedbiomed INFO - message received:{'researcher_id': 'researcher_46a0da2b-2e99-4316-b964-b467d343432b', 'job_id': '73fa8b0c-f733-4362-a17c-a1c447c38dc8', 'success': True, 'client_id': 'client_2a34fb5d-7c54-469f-b034-51062b388dc1', 'dataset_id': 'dataset_655ec19c-3a4c-4b23-9e39-c91deba0a3e3', 'params_url': 'http://localhost:8844/media/uploads/2021/09/30/node_params_9267c53f-e1a5-42c3-bf67-dd37544b7206.pt', 'timing': {'rtime_training': 41.86320511903614, 'ptime_training': 134.356822922}, 'msg': '', 'command': 'train'}
2021-09-30 13:44:54,542 fedbiomed INFO - message received:{'researcher_id': 'researcher_46a0da2b-2e99-4316-b964-b467d343432b', 'job_id': '1f170a81-0c59-4010-a5c7-a8db1b75030f', 'success': True, 'client_id': 'client_2a34fb5d-7c54-469f-b034-51062b388dc1', 'dataset_id': 'dataset_655ec19c-3a4c-4b23-9e39-c91deba0a3e3', 'params_url': 'http://localhost:8844/media/uploads/2021/09/30/node_params_2a2a8592-d623-4e22-8c1e-a869ea601692.pt', 'timing': {'rtime_train

2021-09-30 15:40:50,299 fedbiomed INFO - message received:{'researcher_id': 'researcher_46a0da2b-2e99-4316-b964-b467d343432b', 'success': True, 'databases': [{'name': 'MNIST', 'data_type': 'default', 'tags': ['#MNIST', '#dataset'], 'description': 'MNIST database', 'shape': [60000, 1, 28, 28], 'dataset_id': 'dataset_655ec19c-3a4c-4b23-9e39-c91deba0a3e3'}], 'count': 1, 'client_id': 'client_2a34fb5d-7c54-469f-b034-51062b388dc1', 'command': 'search'}
2021-09-30 15:41:24,688 fedbiomed INFO - message received:{'researcher_id': 'researcher_46a0da2b-2e99-4316-b964-b467d343432b', 'success': True, 'databases': [{'name': 'MNIST', 'data_type': 'default', 'tags': ['#MNIST', '#dataset'], 'description': 'MNIST database', 'shape': [60000, 1, 28, 28], 'dataset_id': 'dataset_da9a5db6-9dda-475e-b920-43f8d1f54377'}], 'count': 1, 'client_id': 'client_b1afd615-e4d4-489f-9ded-bd8db8ff2137', 'command': 'search'}
2021-09-30 15:41:24,697 fedbiomed INFO - message received:{'researcher_id': 'researcher_46a0da2b-2

2021-09-30 15:47:07,982 fedbiomed INFO - message received:{'researcher_id': 'researcher_46a0da2b-2e99-4316-b964-b467d343432b', 'job_id': '8473a6c0-7c4b-4b46-8ef8-6eddd0567860', 'success': True, 'client_id': 'client_b1afd615-e4d4-489f-9ded-bd8db8ff2137', 'dataset_id': 'dataset_da9a5db6-9dda-475e-b920-43f8d1f54377', 'params_url': 'http://localhost:8844/media/uploads/2021/09/30/node_params_509d9977-5b4a-44a6-818a-3dab2922b1f5.pt', 'timing': {'rtime_training': 28.520599263953045, 'ptime_training': 104.34573408200049}, 'msg': '', 'command': 'train'}
2021-09-30 15:47:43,783 fedbiomed INFO - message received:{'researcher_id': 'researcher_46a0da2b-2e99-4316-b964-b467d343432b', 'job_id': '8473a6c0-7c4b-4b46-8ef8-6eddd0567860', 'success': True, 'client_id': 'client_2a34fb5d-7c54-469f-b034-51062b388dc1', 'dataset_id': 'dataset_655ec19c-3a4c-4b23-9e39-c91deba0a3e3', 'params_url': 'http://localhost:8844/media/uploads/2021/09/30/node_params_c55f6297-1c23-45e1-9bd5-7bcd3b3261aa.pt', 'timing': {'rtime

2021-09-30 16:22:06,651 fedbiomed INFO - message received:{'researcher_id': 'researcher_46a0da2b-2e99-4316-b964-b467d343432b', 'job_id': '6f9a5070-b15f-4d7c-89c8-38d4ab2e9397', 'success': True, 'client_id': 'client_b1afd615-e4d4-489f-9ded-bd8db8ff2137', 'dataset_id': 'dataset_da9a5db6-9dda-475e-b920-43f8d1f54377', 'params_url': 'http://localhost:8844/media/uploads/2021/09/30/node_params_9ddca526-5997-45ac-9127-d22f76316c1b.pt', 'timing': {'rtime_training': 29.734978054999374, 'ptime_training': 106.59343688000081}, 'msg': '', 'command': 'train'}
2021-09-30 16:22:42,537 fedbiomed INFO - message received:{'researcher_id': 'researcher_46a0da2b-2e99-4316-b964-b467d343432b', 'job_id': '6f9a5070-b15f-4d7c-89c8-38d4ab2e9397', 'success': True, 'client_id': 'client_2a34fb5d-7c54-469f-b034-51062b388dc1', 'dataset_id': 'dataset_655ec19c-3a4c-4b23-9e39-c91deba0a3e3', 'params_url': 'http://localhost:8844/media/uploads/2021/09/30/node_params_cb17c7b3-57b9-4dd2-a2e6-d9326a351c92.pt', 'timing': {'rtime

2021-09-30 17:10:07,716 fedbiomed INFO - message received:{'researcher_id': 'researcher_46a0da2b-2e99-4316-b964-b467d343432b', 'success': True, 'databases': [{'name': 'MNIST', 'data_type': 'default', 'tags': ['#MNIST', '#dataset'], 'description': 'MNIST database', 'shape': [60000, 1, 28, 28], 'dataset_id': 'dataset_655ec19c-3a4c-4b23-9e39-c91deba0a3e3'}], 'count': 1, 'client_id': 'client_2a34fb5d-7c54-469f-b034-51062b388dc1', 'command': 'search'}
2021-09-30 17:10:38,926 fedbiomed INFO - message received:{'researcher_id': 'researcher_46a0da2b-2e99-4316-b964-b467d343432b', 'job_id': '9f89fedc-00f8-4fc8-8a89-8cf42304255f', 'success': True, 'client_id': 'client_b1afd615-e4d4-489f-9ded-bd8db8ff2137', 'dataset_id': 'dataset_da9a5db6-9dda-475e-b920-43f8d1f54377', 'params_url': 'http://localhost:8844/media/uploads/2021/09/30/node_params_c55ae905-dcc8-413e-8954-0489e23aae50.pt', 'timing': {'rtime_training': 20.602918353048153, 'ptime_training': 72.24496754200027}, 'msg': '', 'command': 'train'}

Local training results for each round and each node are available in `exp.training_replies` (index 0 to (`rounds` - 1) ).

For example you can view the training results for the last round below.

Different timings (in seconds) are reported for each dataset of a node participating in a round :
- `rtime_training` real time (clock time) spent in the training function on the node
- `ptime_training` process time (user and system CPU) spent in the training function on the node
- `rtime_total` real time (clock time) spent in the researcher between sending the request and handling the response, at the `Job()` layer

In [ ]:
print("\nList the training rounds : ", exp.training_replies.keys())

print("\nList the clients for the last training round and their timings : ")
round_data = exp.training_replies[rounds - 1].data
for c in range(len(round_data)):
    print("\t- {id} :\
    \n\t\trtime_training={rtraining:.2f} seconds\
    \n\t\tptime_training={ptraining:.2f} seconds\
    \n\t\trtime_total={rtotal:.2f} seconds".format(id = round_data[c]['client_id'],
        rtraining = round_data[c]['timing']['rtime_training'],
        ptraining = round_data[c]['timing']['ptime_training'],
        rtotal = round_data[c]['timing']['rtime_total']))
print('\n')
    
exp.training_replies[rounds - 1].dataframe

Federated parameters for each round are available in `exp.aggregated_params` (index 0 to (`rounds` - 1) ).

For example you can view the federated parameters for the last round of the experiment :

In [ ]:
print("\nList the training rounds : ", exp.aggregated_params.keys())

print("\nAccess the federated params for the last training round :")
print("\t- params_path: ", exp.aggregated_params[rounds - 1]['params_path'])
print("\t- parameter data: ", exp.aggregated_params[rounds - 1]['params'].keys())


## Optional : searching the data

In [ ]:
from fedbiomed.researcher.requests import Requests

r = Requests()
data = r.search(tags)

import pandas as pd
for client_id in data.keys():
    print('\n','Data for ', client_id, '\n\n', pd.DataFrame(data[client_id]))

## Optional : clean file repository (do not run unless necessary)
Clean all the files in the repo via the rest API.

In [ ]:
# import requests
# from fedbiomed.researcher.environ import UPLOADS_URL

# uploaded_models = requests.get(UPLOADS_URL).json()
# for m in uploaded_models:
#   requests.delete(m['url'])

Feel free to try your own models :D